<a href="https://colab.research.google.com/github/debypivatto/my-project/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Projeto Final
Tema: E-Commerce

1. Objetivo geral do projeto: Verificar perfis de usuários com base no consumo da loja Everything Plus, uma loja online que vende utensílios domésticos

2. Entendimento do problema e entregas esperadas:
Problema: Prever comportamentos de clientes atraves dos seus perfis de consumo , utilizando análise exploratória de dados, segmentação por histórico de compras e testes de hipótese e desenvolver com a equipe de marketing o desenvolvimento de promoções

3. Coleta e Preparação dos Dados:

Informação sobre os Dados:
`InvoiceNo` — identificador de pedido
`StockCode` — identificador de item
`Description` — nome de item
`Quantity`
`InvoiceDate` — data do pedido
`UnitPrice` — preço por item
`CustomerID`

3.1 Importar os dados
Carregar todos o arquivo e verificar dados básicos e gerais para poder realizar o tratamento dos dados (.info, .head etc)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv('ecommerce_dataset_us.csv', sep='\t')

In [ ]:
df.head()

In [ ]:
df.info(memory_usage='deep') # observando tipo de dados

3.2 Tratar os dados - Limpeza e Pré-processamento

In [ ]:
df.columns

In [ ]:
# Percorrendo os cabeçalhos e convertendo tudo em minúsculas
df.columns = df.columns.str.lower()

In [ ]:
df.columns # verificando se converteu

In [ ]:
#Verificar valores ausentes e substituir valores ausentes (se necessário)
df.isnull().sum()

In [ ]:
df['description'].describe()

In [ ]:
df['description'].nunique()

In [ ]:
df['description'].fillna('unknown', inplace=True) # retirando os nomes em branco e substituindo por desconhecido

In [ ]:
df['description'].isnull().sum() # verificando se removeu

In [ ]:
df['description'].duplicated().sum()

In [ ]:
df['description'].drop_duplicates().reset_index(drop=True).to_frame(name='item')

In [ ]:
df['customerid'].describe()

In [ ]:
df['customerid'].isnull().sum()

In [ ]:
df['customerid'].nunique() # 4372 clientes unicos

In [ ]:
df['customerid'].fillna('unknownID', inplace=True) # retirando os nomes em branco e substituindo por desconhecido

In [ ]:
df['customerid'].isnull().sum() #verificando de converteu

In [ ]:
df['customerid'].duplicated().sum() #Verificar valores duplicados e remover duplicados (se necessário)

In [ ]:
df['customerid'].drop_duplicates()

In [ ]:
#Converter tipos para otimizar espaço do arquivo e facilitar análise (se necessário)
df['invoicedate'] = pd.to_datetime(df['invoicedate'])

In [ ]:
df.info()

In [ ]:
#Converter variáveis categóricas e normalizar valores numéricos
df['invoiceno'] = pd.to_numeric(df['invoiceno'], errors='coerce').astype('Int64')
df['stockcode'] = pd.to_numeric(df['stockcode'], errors='coerce').astype('Int64')
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce').astype('Int64')

In [ ]:
df.info(memory_usage='deep') # observando tipo de dados, converteu tudo para inteiro o orquivo ficou mais leve com metade do tamanho inicial

In [ ]:
#Verificar consistência e unicidade de customerID
df['customerid'].describe() # nao posso passar para inteiro pois existe uma grande quantidade de usuarios desconhecidos que nao podemos ignorar na analise

In [ ]:
#Verificar integridade dos dados antes de seguir (conferindo numero de linhas etc)
df.sample(10)

In [ ]:
df['invoiceno'].sample(3)

In [ ]:
df['stockcode'].sample(3)

4. Analise Exploratoria dos Dados

4. Análise Exploratória dos Dados:
4.1 Analisar os dados(distribuições, correlações e outliers) explorando correlações entre variáveis

In [ ]:
#investigar o porque de valores negativos, provavelmente ligados a devolucoes de estoque.

In [ ]:
# Criar coluna de valor total da compra
df['totalprice'] = df['quantity'] * df['unitprice']

In [ ]:
df['totalprice'].describe()

In [ ]:
# esse numero minimo negativo deve indicar o numero de devolucoes, devemos excluir quando analisar os dados

In [ ]:
df

In [ ]:
df[df['quantity'] < 0].sample(5)
#investigar o porque de valores negativos, provavelmente ligados a devolucoes de estoque

In [ ]:
#preciso remover esses valores pois pretendo analisar as compras que sao valores positivos, então removo todos os valores negativos que provavelmente são as devoluções
df_vendas = df[(df['quantity'] > 0) & (df['unitprice'] > 0)].copy()

In [ ]:
df_vendas #data frame sem valores negativos

In [ ]:
top10 = df_vendas.groupby('description')['quantity'].sum().sort_values(ascending=False).head(10) #top produtos vendidos em quantidades vendidas - unidades

In [ ]:
plt.figure(figsize=(8,6))
sns.barplot(x=top10.values, y=top10.index)
plt.title('Top 10 Prudutos por Unidades')
plt.xlabel('Quantidade de Produtos')
plt.ylabel('Nome do Produto')
plt.show() # Criando gráfico

In [ ]:
topproducts = df_vendas['description'].value_counts().head(10) #top produtos gerais, os que mais aparecem

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=topproducts.values, y=topproducts.index)
plt.title('Top 10 Prudutos')
plt.xlabel('Quantidade de Produtos')
plt.ylabel('Nome do Produto')
plt.show()

In [ ]:
top10unit = df_vendas.groupby('description')['unitprice'].sum().sort_values(ascending=False).head(10)  #top10 produtos vendidos por preço unitãrio

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=top10unit.values, y=top10unit.index)
plt.title('Top 10 Produtos por Preco Unitario')
plt.xlabel('Quantidade de Produtos')
plt.ylabel('Nome do Produto')
plt.show()

In [ ]:
top10total = df_vendas.groupby('description')['totalprice'].sum().sort_values(ascending=False).head(10)  #top10 produtos vendidos em preco total do pedido

In [ ]:
top10total.plot(kind='bar', figsize=(22, 6), color='purple', title='Top 10 Produtos Mais Vendidos em Valores Totais')
# Ajustar rótulos
plt.xlabel('Nome do Produto')
plt.ylabel('Vendas Totais (em mil)')
plt.xticks(rotation=45)  # Inclina os nomes pra melhor leitura
plt.show()

In [ ]:
top10clients = df_vendas.groupby('customerid')['totalprice'].sum().sort_values(ascending=False).head(10) #top 10 clientes por venda total, no caso soma de venda de usuarios desconhecidos nao pode ser ignorada, focar em identificar para poder segmentar melhor
print (top10clients)

In [ ]:
top10clients.plot(kind='bar', figsize=(12, 6), color='orange', title='Top 10 Clientes em Valores Totais de Pedidos')
# Ajustar rótulos
plt.xlabel('Cliente')
plt.ylabel('Vendas Totais (em mil)')
plt.xticks(rotation=90)  # Inclina os nomes pra melhor leitura
plt.show()

In [ ]:
df_vendas.groupby('customerid')['totalprice'].describe()

In [ ]:
dfct = df_vendas.groupby('customerid')['totalprice'].mean().reset_index()
print (dfct)

In [ ]:
plt.figure(figsize=(8, 6))
plt.boxplot(dfct['totalprice'], vert=True)
plt.title('Boxplot: Média de Valor dos Pedidos por Cliente')
plt.xlabel('Média do Valor dos Pedidos')
plt.tight_layout()
plt.show()

In [ ]:
tops = df_vendas.groupby('customerid')['description'].value_counts().nlargest(5) #top
print (tops)

In [ ]:
pedidospordia = df_vendas.groupby(df['invoicedate'].dt.date)['invoiceno'].nunique()
print (pedidospordia)

In [ ]:
pedidospordia.plot(title='Número de pedidos por dia')
plt.xlabel("Data")
plt.ylabel("Pedidos únicos") #pedidos por dia

In [ ]:
df_vendas['invoicedate'].describe() # dados entre 29 de out 2018 e 7 de dez 2019  - aproximadamente 1 ano e 3 meses de análises, top de vendas foi em 29 de out de 2019

In [ ]:
clientespordia = df_vendas.groupby(df['invoicedate'].dt.date)['customerid'].nunique()
clientespordia.plot(title='Número de clientes por dia')
plt.xlabel("Data")
plt.ylabel("Clientes") # clientes por dia

In [ ]:
vendaspordia = df_vendas.groupby(df['invoicedate'].dt.date)['totalprice'].sum()
vendaspordia.plot(title='Total de vendas por dia (USD)')
plt.xlabel("Data")
plt.ylabel("Valor de Vendas") #total de vendas por dia

In [ ]:
vendaspordia.describe()

In [ ]:
df_vendas['date'] = df_vendas['invoicedate'].dt.date
df_vendas['hour'] = df_vendas['invoicedate'].dt.hour

In [ ]:
df_vendas['date'].describe() # maior dia de vendas 03/12/2019

In [ ]:
df_vendas['hour'].describe() # entre as 6 e as 20 hrs, sendo que maior parte dos pedidos fica entre as 13 e 15 hrs.

Grafico de Dispersao
Produtos e Valor total

In [ ]:
df_vp = df_vendas.groupby('description')['totalprice'].sum().reset_index()
print (df_vp)

In [ ]:
df_top20 = df_vp.nlargest(30, 'totalprice')
# Plotar gráfico de dispersão
plt.figure(figsize=(20, 6))
plt.scatter(df_top20['description'], df_top20['totalprice'], color='green')
plt.xticks(rotation=90)
plt.title('Top 30 Produtos por Valor Total de Venda')
plt.xlabel('Produto (description)')
plt.ylabel('Valor Total (totalprice)')
plt.tight_layout()
plt.show()

In [ ]:
df_vendas['totalprice'].describe() # maioria dos pedidos fica entre 9.90 e 17.70 dolares.

In [ ]:
df_vendas['unitprice'].describe() # maioria dos precos unitarios fica entre 2 e 4 dolares.

In [ ]:
heatmap_data = df_vendas.groupby(['date', 'hour'])['totalprice'].sum().unstack()

In [ ]:
# Converte todos os valores para float
heatmap_data = heatmap_data.apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)

In [ ]:
df_vendas['week'] = pd.to_datetime(df_vendas['invoicedate']).dt.to_period('W').apply(lambda r: r.start_time)
df_vendas['hour'] = pd.to_datetime(df_vendas['invoicedate']).dt.hour
df_vendas['week'] = df_vendas['week'].dt.strftime('%d/%m/%Y')
heatmap_data = df_vendas.groupby(['week', 'hour'])['totalprice'].sum().unstack()
heatmap_data = heatmap_data.fillna(0).astype(float)

In [ ]:
plt.figure(figsize=(18, 8))
sns.heatmap(heatmap_data, cmap='Oranges', linewidths=2, linecolor='white')
plt.title("Heatmap: Vendas Semanais por Hora do Dia", fontsize=16)
plt.xlabel("Hora do Dia")
plt.ylabel("Semana")
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

Segmentação com K-means (Machine Learning)

In [ ]:
# Pré-processamento
X = dfct[['totalprice']].copy()

# Aplica K-means para 3 segmentos (Baixo, médio, alto)
kmeans = KMeans(n_clusters=3, random_state=42)
dfct['segmento_kmeans'] = kmeans.fit_predict(X)

# Opcional: ordenar os clusters do menor para o maior valor médio
dfct['segmento_kmeans'] = dfct['segmento_kmeans'].map(
    dict(sorted(zip(
        np.argsort(kmeans.cluster_centers_.flatten()),
        ['Baixo', 'Médio', 'Alto'] ))))

In [ ]:
# Ver exemplo do resultado aleatorio de amostras
print(dfct[['customerid', 'totalprice', 'segmento_kmeans']].sample(10))

In [ ]:
plt.figure(figsize=(18, 12))
sns.boxplot(data=dfct, x='segmento_kmeans', y='totalprice', palette='Oranges')
plt.title('Distribuição dos Segmentos de Clientes (K-means)')
plt.xlabel('Segmento')
plt.ylabel('Média de Valor dos Pedidos')
plt.show()

In [ ]:
dfct

4.4 Validação e testes
    Avaliar métricas (precision, recall, F1-score)

In [ ]:
# Definir X e Y
X = dfct.drop(columns=['totalprice', 'segmento_kmeans'])
y = dfct['segmento_kmeans']
X = pd.get_dummies(X, drop_first=True)

# Dividir
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Classificador
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_valid)
print("Floresta Aleatória (Classificação de segmento):")
print(classification_report(y_valid, y_pred_rf))

In [ ]:
feat_importance = pd.Series(rf_model.feature_importances_, index=X.columns)
feat_importance.nlargest(10).plot(kind='barh')
plt.title("Top 10 Variáveis Importantes")

Previsões e Conclusões:
Recomendações para a equipe de marketing com os achados

Produtos top 5 - Fazer ações específicas para esses produtos que são o carro chefe da loja, por exemplo na compra de 4 leve 1 gratuitamente. Tendência é aumentar a venda desses items.
Produtos que mais vendem - valor unitário entre 2 a 4 dólares, focar nesse tipo de produto que apesar de ter baixo agrega maior valor total, aumentando ticket médio.
A maioria dos pedidos fica entre 9.90 e 17.70 dolares, fazer promoções tentando aumentar esse valor médio de pedidos, por exemplo se pedido ultrapassar 20 dólares oferecer uma amostra ou brinde ou desconto na próxima compra.
Média de pedidos por dia - até 140 pedidos, tentar aumentar esses valores de pedidos com alertas por email e SMS para promoções no segundo pedido da semana ou dia com descontos.
Média de 120 clientes por dia - alguns repetem mais de 1 pedido ao dia, tentar aumentar esse número fazendo comprarem por repetição, oferecer frete grátis pro segundo ou terceiro pedido no mesmo dia, já que serão entregues no mesmo momento.
Dados da análise entre 29 de out 2018 e 7 de dez 2019  - aproximadamente 1 ano e 3 meses de análises, top de vendas foi em 29 de out de 2019 - tentar entender se houve algum evento para esse elevado número de vendas.
Tentar saber e forçar registro dos clientes unkown visto que suas compras são consideráveis e por isso não podemos descartá-los da análise, tentar de alguma forma identificar o cliente para poder estudar melhor o perfil desses clientes seja através de telefone ou email.
Hora dos pedidos - entre 9 e 17 hrs maior número de pedidos, instalar pop-ups que nesses horários tentem aumentar venda de items por impulso.
A maior parte dos clientes e os que realmente impactam a lucratividade são os que realizam pedidos mais baixos porém continuamente, valorizar fidelização, sistema de pontos e descontos etc.

In [ ]:
# Exporta para CSV para fazer tableau
df_vendas.to_csv('vendas.csv', index=False)

Dashbords  - links tableau publico
https://public.tableau.com/app/profile/debora.pivatto/viz/FinalProject1_17541317175330/Sheet1?publish=yes
https://public.tableau.com/app/profile/debora.pivatto/viz/FinalProject1_1_17541324490910/Sheet2?publish=yes

Fontes e Funções:
Pandas -  Manipulação de DataFrames, agregações
Seaborn -  Criação de gráficos estatísticos
Tableau Public - Criação de dashboards interativos
Matplotlib - Gráficos personalizados
scikit-learn documentation - KMeans, StandardScaler, métricas

In [ ]:
Apresentacao - ppt
file:///C:/Users/User/Tripleten/Sprint%2014/Presentation.pdf
Anexado neste notebook
https://1drv.ms/p/c/0d14907ef08d7971/EThdrNeP_T5LkE7w6ocVtmEBKN3CiAujTrp7dy8C_YyOiw?e=clUXCO